# Example Ordering

In this example notebook, we will repeat the operations of the Example Operations notebook and time them:

1. Locate the entries in an STM which intersect building polygons;
2. Add year of construction as an attribute to the STM.

Then we will order the STM according to the Morton code of the pixel coordinates and repeat these operations, again timing them.

Finally, we will visualize the change in the order of the elements and compare the processing times.

## Repeat example operations 

For setup and more notes on the operations, see [Example Operations notebook](./demo_operations_stm.ipynb)

Note that this can take a few minutes.

In [1]:
from pathlib import Path
import xarray as xr
import numpy as np
import stmtools

path_stm = Path('./stm.zarr')
#chunksize = 10000
chunksize = 500
path_polygon = Path('bag_light_AMS_WGS84.gpkg')
fields_to_query = ['bouwjaar']

In [ ]:
# Example operations on original STM.
stmat = xr.open_zarr(path_stm)
stmat = stmat.chunk({"space": chunksize, "time": -1})
stmat_subset = stmat.stm.subset(method='polygon', polygon=path_polygon)
stmat_enriched = stmat_subset.stm.enrich_from_polygon(path_polygon, fields_to_query)
year_construction = stmat_enriched['bouwjaar'].compute()

/storage/MobyLe/stmtools/stmtools/stm.py:134: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  if dim not in self._obj.dims.keys():


In [ ]:
print(stmat)

In [ ]:
print(stmat_subset)

### Compute timing statistics for the operations

Note that these timing tests were not performed immediately, because the ```%timeit``` magic function does not allow variable assignment.

In [ ]:
# Compute timings of operations on original STM.
time_subset = %timeit -o stmat.stm.subset(method='polygon', polygon=path_polygon)
time_enrich = %timeit -o stmat_subset.stm.enrich_from_polygon(path_polygon, fields_to_query)
time_enrich_compute = %timeit -o stmat_enriched['bouwjaar'].compute()

## Reorder STM and repeat operations

In [ ]:
# Reorder STM.
stmat_ordered = stmat.stm.reorder(xlabel="azimuth", ylabel="range")

In [ ]:
## stmat_ordered = stmat_ordered.unify_chunks()
#stmat_ordered_chunked = stmat_ordered.chunk({"space": 10000, "time":-1})
#stmat_ordered_chunked

In [ ]:
# Repeat example operations on ordered STM.
stmat_ordered_subset = stmat_ordered.stm.subset(method='polygon', polygon=path_polygon)
stmat_ordered_enriched = stmat_ordered_subset.stm.enrich_from_polygon(path_polygon, fields_to_query)
year_construction = stmat_enriched['bouwjaar'].compute()

In [ ]:
print(stmat_ordered)

In [ ]:
print(stmat_ordered_subset)

### Compute timing statistics for the operations above.

In [ ]:
# Compute timings of ordering STM.
time_ordering = %timeit -o stmat.stm.reorder(xlabel="azimuth", ylabel="range")

In [ ]:
# Compute timings of operations on ordered STM.
time_ordered_subset = %timeit -o stmat_ordered.stm.subset(method='polygon', polygon=path_polygon)
time_ordered_enrich = %timeit -o stmat_ordered_subset.stm.enrich_from_polygon(path_polygon, fields_to_query)
time_ordered_enrich_compute = %timeit -o stmat_ordered_enriched['bouwjaar'].compute()

## Visualize the results

The images below are colored by element index.

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.cm as cm

In [ ]:
# Visualize original results.
fig, ax = plt.subplots()
plt.title("Element index, original")
plt.scatter(stmat_enriched.lon.data, stmat_enriched.lat.data, c=np.arange(len(stmat_enriched.lon)), s=0.004, cmap=cm.jet)
plt.colorbar()

In [ ]:
# Visualize ordered results.
fig, ax = plt.subplots()
plt.title("Element index, ordered")
plt.scatter(stmat_ordered_enriched.lon.data, stmat_ordered_enriched.lat.data, c=np.arange(len(stmat_ordered_enriched.lon)), s=0.004, cmap=cm.jet)
plt.colorbar()

### Compare operation times

In [ ]:
print(f"Subset (original):  {time_subset}")
print(f"Enrich (original):  {time_enrich}")
print(f"Compute (original): {time_enrich_compute}")

In [ ]:
print(f"Ordering:           {time_ordering}")

In [ ]:
print(f"Subset (ordered):   {time_ordered_subset}")
print(f"Enrich (ordered):   {time_ordered_enrich}")
print(f"Compute (ordered):  {time_ordered_enrich_compute}")

In [ ]:
print(f"Subset (diff):      {time_subset.average-time_ordered_subset.average}")
print(f"Enrich (diff):      {time_enrich.average-time_ordered_enrich.average}")
print(f"Compute (diff):     {time_enrich_compute.average-time_ordered_enrich_compute.average}")